This Notebook is a kaggle tutorial for Japanese kaggle beginners writen in Japanese.

# 5. 機械学習アルゴリズムのお気持ち？！ ハイパーパラメータを調整してみよう

先にも説明したように、機械学習アルゴリズムの振る舞いはハイパーパラメータという値で制御されます。もちろん、ハイパーパラメータの値次第で予測結果は変わり得ます。

ハイパーパラメータの調整は、主に2種類の方法があります。

- 手動で調整
- チューニングツールを使う

後者としては、[Grid search](https://qiita.com/yhyhyhjp/items/c81f7cea72a44a7bfd3a), [Bayesian Optimization](https://blog.amedama.jp/entry/2018/08/18/233841), [Hyperopt](https://blog.amedama.jp/entry/hyperopt), [Optuna](https://research.preferred.jp/2018/12/optuna-release/)など、いくつかのツールがあります。

この[Notebook](https://www.kaggle.com/sishihara/upura-kaggle-tutorial-05-tuning)では、最初に手動でハイパーパラメータを調整し、機械学習アルゴリズムの振る舞いが異なることを確認します。その後、Optunaを用いたチューニングを実施します。

In [ ]:
# 特徴量の準備

import numpy as np
import pandas as pd

train = pd.read_csv("../input/titanic/train.csv")
test = pd.read_csv("../input/titanic/test.csv")
gender_submission = pd.read_csv("../input/titanic/gender_submission.csv")

data = pd.concat([train, test], sort=False)

data['Sex'].replace(['male','female'], [0, 1], inplace=True)
data['Embarked'].fillna(('S'), inplace=True)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data['Fare'].fillna(np.mean(data['Fare']), inplace=True)
data['Age'].fillna(data['Age'].median(), inplace=True)
data['FamilySize'] = data['Parch'] + data['SibSp'] + 1
data['IsAlone'] = 0
data.loc[data['FamilySize'] == 1, 'IsAlone'] = 1

In [ ]:
data.head()

In [ ]:
delete_columns = ['Name', 'PassengerId', 'Ticket', 'Cabin']
data.drop(delete_columns, axis=1, inplace=True)

train = data[:len(train)]
test = data[len(train):]

y_train = train['Survived']
X_train = train.drop('Survived', axis=1)
X_test = test.drop('Survived', axis=1)

In [ ]:
X_train.head()

特徴量の準備が完了しました。

## LightGBM

X_trainをX_train（学習用）とX_valid（検証用）に分割します。

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0, stratify=y_train)

In [ ]:
# カテゴリ変数の指定
categorical_features = ['Embarked', 'Pclass', 'Sex']

# 手動で調整

次の部分で、LightGBMのハイパーパラメータを定義します。前回は `objective` のみを指定していました。明示的に指定しない場合は、 `default` の[値](https://lightgbm.readthedocs.io/en/latest/Parameters.html)が自動的に定義されます。

In [ ]:
params = {
    'objective': 'binary'
}

後の比較のために、このハイパーパラメータで学習・予測を実施しておきます。

In [ ]:
import lightgbm as lgb


lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

ここでは公式documentationの「[Parameters Tuning](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html)」に従って、手動で調整を行っていきましょう。いくつかのユースケース別に、ハイパーパラメータ調整のTipsが記載されています。

今回は、精度を高めるのが目的なので「[For Better Accuracy](https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html#for-better-accuracy)」を参照します。

> - Use large max_bin (may be slower)
> - Use small learning_rate with large num_iterations
> - Use large num_leaves (may cause over-fitting)
> - Use bigger training data
> - Try dart

- 1つ目は「大きめの`max_bin`を使え」です。`default`の値は255なので、ここでは300にしてみます。
- 2つ目は「小さめの`learning_rate`を使え」です。`default`の値は0.1なので、ここでは0.05にしてみます。
- 3つ目は「大きめの`num_leaves`を使え」です。`default`の値は31なので、ここでは40にしてみます。

手動で調整するにせよ、チューニングツールを使うにせよ、機械学習アルゴリズムをブラックボックス的に利用するのではなく、ハイパーパラメータを正しく理解することが非常に大切です。

ハイパーパラメータの説明については、英語ですが[公式のdocumentation](https://lightgbm.readthedocs.io/en/latest/Parameters.html)で確認するのが確実です。なお日本語の記事だと、例えば[こちら](https://nykergoto.hatenablog.jp/entry/2019/03/29/%E5%8B%BE%E9%85%8D%E3%83%96%E3%83%BC%E3%82%B9%E3%83%86%E3%82%A3%E3%83%B3%E3%82%B0%E3%81%A7%E5%A4%A7%E4%BA%8B%E3%81%AA%E3%83%91%E3%83%A9%E3%83%A1%E3%83%BC%E3%82%BF%E3%81%AE%E6%B0%97%E6%8C%81%E3%81%A1)にLightGBMなどの勾配ブースティングの主要なハイパーパラメータ解説が記載されています。

In [ ]:
params = {
    'objective': 'binary',
    'max_bin': 300,
    'learning_rate': 0.05,
    'num_leaves': 40
}

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred[:10]

y_predがハイパーパラメータ変更前と異なる値を取っていると分かります。出力ログにも変化があり、最終的な`valid_1's binary_logloss`が 0.433251 と、変更前よりも小さい値になっています。`binary_logloss` は損失なので、小さい方が望ましいです。

In [ ]:
y_pred = (y_pred > 0.5).astype(int)
y_pred[:10]

In [ ]:
sub = gender_submission

sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm_handtuning.csv", index=False)

sub.head()

LightGBMでの予測結果を提出してみると、私の環境では0.77033というスコアが出ました。ハイパーパラメータ変更前の0.75598に比べて、スコアが向上しています。

# Optunaを使う

ここまで手動でハイパーパラメータを調整してきましたが、次のような感情が芽生えている方もいるのではないでしょうか。

- 「大きめ」「小さめ」といっても、具体的にどの値にすればよいのか分からない
- 各パラメータの組み合わせ方もいくつかあり、逐一設定・実行して性能を検証するのは煩わしい

そのような課題を解決してくれるのが、ハイパーパラメータのチューニングツールです。今回はOptunaを使っていきます。

Optunaを使うに当たっては、あらかじめ次の関数の`trial.suggest_int()`のように、探索範囲を定義します。 

ここでは、意図的に`learning_rate`の調整を実施していません。テーブルデータをLightGBMで扱う場合、一般に`learning_rate`が低いほど高い性能が得られるためです。そのため探索範囲には含めず、必要であれば後に手動で低い値に変更することにします。

In [ ]:
# KaggleのKernelに搭載済
import optuna
from sklearn.metrics import log_loss


def objective(trial):
    params = {
        'objective': 'binary',
        'max_bin': trial.suggest_int('max_bin', 255, 500),
        'learning_rate': 0.05,
        'num_leaves': trial.suggest_int('num_leaves', 32, 128),
    }
    
    lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

    model = lgb.train(
        params, lgb_train,
        valid_sets=[lgb_train, lgb_eval],
        verbose_eval=10,
        num_boost_round=1000,
        early_stopping_rounds=10
    )

    y_pred_valid = model.predict(X_valid, num_iteration=model.best_iteration)
    score = log_loss(y_valid, y_pred_valid)
    return score

`n_trials`は試行回数です。ここでは計算を短くするため、40回程度にしておきます。乱数も固定しておきます。

In [ ]:
study = optuna.create_study(sampler=optuna.samplers.RandomSampler(seed=0))
study.optimize(objective, n_trials=40)

In [ ]:
study.best_params

指定した範囲内で試行回数だけ探索した結果得られた最良のハイパーパラメータが表示されています。こちらで改めて予測し直して、提出してみましょう。

In [ ]:
params = {
    'objective': 'binary',
    'max_bin': study.best_params['max_bin'],
    'learning_rate': 0.05,
    'num_leaves': study.best_params['num_leaves']
}

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

In [ ]:
y_pred = (y_pred > 0.5).astype(int)

sub['Survived'] = y_pred
sub.to_csv("submission_lightgbm_optuna.csv", index=False)

sub.head()

予測結果を提出してみると、私の環境では0.77033というスコアが出ました。偶然手動での調整と同じスコアになっています。探索範囲や試行回数を変えれば、より良いスコアが出るかもしれません。